<a href="https://colab.research.google.com/github/pedrobribeiro2/99Hunters_Projects/blob/main/2023%2012%20Retrospectiva%20Companies_Processamento%20de%20Reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Adjust the file path below to the location of your file in Google Drive
file_path = '/content/drive/MyDrive/Pedro Marketing/2023 12 99Hunters Wrapped/2023 12 05 jobs_report.csv'  # Replace with your file path in Google Drive
candidates_data_path = '/content/drive/MyDrive/Pedro Marketing/2023 12 99Hunters Wrapped/2023 12 05 candidate_jobs_report.csv'

# Load the dataset
jobs_data = pd.read_csv(file_path)
candidates_data = pd.read_csv(candidates_data_path)

# Display the first few rows to verify the data
#print(candidates_data.head())


In [ ]:
#print(jobs_data[['Company_name', 'Created_at', 'been_open', 'been_shortlist']])

In [ ]:
# Importar a biblioteca necessária
import calendar

#Converter as colunas de data para datetime
jobs_data['Created_at'] = pd.to_datetime(jobs_data['Created_at'], format='%m/%d/%Y-%H:%M')
jobs_data['been_open'] = pd.to_datetime(jobs_data['been_open'], format='%m/%d/%Y-%H:%M', errors='coerce')
jobs_data['been_working'] = pd.to_datetime(jobs_data['been_working'], format='%m/%d/%Y-%H:%M')
jobs_data['been_shortlist'] = pd.to_datetime(jobs_data['been_shortlist'], format='%m/%d/%Y-%H:%M', errors='coerce')
jobs_data['been_Interviews'] = pd.to_datetime(jobs_data['been_Interviews'], format='%m/%d/%Y-%H:%M')
jobs_data['been_adv_interviews'] = pd.to_datetime(jobs_data['been_adv_interviews'], format='%m/%d/%Y-%H:%M')
jobs_data['been_closed'] = pd.to_datetime(jobs_data['been_closed'], format='%m/%d/%Y-%H:%M', errors='coerce')
jobs_data['been_concluded'] = pd.to_datetime(jobs_data['been_concluded'], format='%m/%d/%Y-%H:%M', errors='coerce')
jobs_data['been_on_hold'] = pd.to_datetime(jobs_data['been_on_hold'], format='%m/%d/%Y-%H:%M')

In [ ]:
# Filtrar jobs válidos
valid_jobs = jobs_data[~jobs_data['Status'].isin(['denied', 'review', 'canceled']) & (jobs_data['Created_at'] >= '2023-01-01')].copy()
# Excluir registros onde a data de abertura é posterior à data de fechamento ou shortlist
#valid_jobs = valid_jobs[(valid_jobs['been_closed'] >= valid_jobs['been_open']) & (valid_jobs['been_shortlist'] >= valid_jobs['been_open'])]


In [ ]:
def classificar_vaga(budget):
    if budget > 15000:
        return 'Executiva'
    else:
        return 'Tradicional'

def estimativa_investimento(budget):
    if budget > 15000:
        return budget * 12 * 0.25  # 25% do pacote anual
    else:
        return budget * 12 * 0.18  # 18% do pacote anual

# Aplicando as funções para classificar as vagas e calcular a estimativa de investimento
jobs_data['Nível Vagas'] = jobs_data['Budget'].apply(classificar_vaga)
jobs_data['Estimativa Investimento'] = jobs_data['Budget'].apply(estimativa_investimento)


In [ ]:
# Calcular a diferença entre as datas de abertura e fechamento em dias
#valid_jobs['Closing_Time'] = (valid_jobs['been_closed'] - valid_jobs['been_open']).dt.days

# Calcular a diferença entre as datas de abertura e envio de shortlist
valid_jobs['SLA_Shortlist'] = ((valid_jobs['been_shortlist'] - valid_jobs['Created_at']).dt.days)
#print(valid_jobs[['Company_name', 'been_open', 'been_shortlist', 'SLA_Shortlist']])


In [ ]:
# Calcular número de jobs abertos, fechados com a 99Hunters, fechados internamente, e total investido
result = valid_jobs.groupby('Company_name').agg(
    Jobs_Abertos=('been_open', 'count'),
    Jobs_Placement=('been_closed', lambda x: x.notna().sum()),
    Jobs_Concluded=('been_concluded', lambda x: x.notna().sum()),
    Total_Investido=('Budget', 'sum'),
    Tempo_Médio_Fechamento=('SLA Closed' , 'mean'),
    Mediana_Fechamento=('SLA Closed' , 'median'),
    #Closing_time=('Closing_Time', 'mean'),
    #Mediana_Fechamento =('Closing_Time', 'median'),
    Média_SLA_Shortlist =('SLA_Shortlist', 'mean'),
    #Mediana_SLA_Shortlist=('SLA_Shortlist', 'median'),
).reset_index()

In [ ]:
#Preparando Candidates_data
candidates_data['Interview_Occurred'] = candidates_data['cj been_interviews'].notna()


In [ ]:
# Contar entrevistas por job
interviews_per_job = candidates_data.groupby('Code')['Interview_Occurred'].sum().reset_index()
interviews_per_job.rename(columns={'Interview_Occurred': 'Número de Entrevistas'}, inplace=True)


In [ ]:
# Cruzamento com a base de jobs
merged_data = jobs_data.merge(interviews_per_job, left_on='code', right_on='Code', how='left')

In [ ]:
# Agregar dados por empresa
agg_data = merged_data.groupby('Company_name').agg(
    Total_Entrevistas=('Número de Entrevistas', 'sum'),
    Total_Jobs=('code', 'nunique')
)
agg_data['Média de Entrevistas por Job'] = agg_data['Total_Entrevistas'] / agg_data['Total_Jobs']

In [ ]:
# Adicionar as novas colunas ao DataFrame resultante
result = result.merge(agg_data[['Total_Entrevistas', 'Média de Entrevistas por Job']], on='Company_name', how='left')


In [ ]:
# Agregar os dados por empresa
agg_data = jobs_data.groupby('Company_name').agg(
    Nível_Vagas=('Nível Vagas', lambda x: list(x.unique())),
    Estimativa_Investimento_Consultoria=('Estimativa Investimento', 'sum')
)

# Adicionar as novas colunas ao DataFrame resultante
result = result.merge(agg_data, on='Company_name', how='left')

# Ajustar o resultado final
result.fillna({'Estimativa_Investimento_Consultoria': 0}, inplace=True)
#result['Estimativa_Investimento_Consultoria'] = result['Estimativa_Investimento_Consultoria'].round(2)
result['Estimativa_Investimento_Consultoria'] = result['Estimativa_Investimento_Consultoria'].apply(lambda x: f'R$ {x:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.'))


In [ ]:
# Tratar valores nulos e arredondar os resultados
result[['Tempo_Médio_Fechamento' , 'Mediana_Fechamento' , 'Média_SLA_Shortlist']] = result[['Tempo_Médio_Fechamento' , 'Mediana_Fechamento' , 'Média_SLA_Shortlist']].fillna(0)#.round(0)


In [ ]:
# Ajustar o resultado final
result.fillna({'Total_Entrevistas': 0, 'Média de Entrevistas por Job': 0}, inplace=True)
result['Média de Entrevistas por Job'] = result['Média de Entrevistas por Job'].round(2)


In [ ]:
# Ajustar formatação de colunas numéricas
colunas_numericas = ['Tempo_Médio_Fechamento' , 'Mediana_Fechamento' , 'Média_SLA_Shortlist', 'Total_Entrevistas','Média de Entrevistas por Job' ]
result[colunas_numericas] = result[colunas_numericas].fillna(0).astype(float).round(2)

# Função para formatar números com vírgula como separador decimal
def format_float(val):
    return f"{val:.2f}".replace('.', ',')

# Aplicar a formatação nas colunas numéricas
result[colunas_numericas] = result[colunas_numericas].applymap(format_float)

# Exibir o DataFrame para verificar as mudanças
#print(result.head())

# Formatar 'Earnings_2023' como moeda BRL
result['Total_Investido'] = result['Total_Investido'].apply(lambda x: f'R$ {x:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.'))


In [ ]:
# Colunas na ordem desejada
colunas_desejadas = [
    'Company_name', 'Jobs_Abertos', 'Jobs_Placement', 'Jobs_Concluded',
    'Total_Investido', 'Nível_Vagas', 'Estimativa_Investimento_Consultoria',
    'Tempo_Médio_Fechamento', 'Mediana_Fechamento', 'Média_SLA_Shortlist',
    'Total_Entrevistas', 'Média de Entrevistas por Job'
]

# Reordenar as colunas de acordo com a nova ordem
result = result[colunas_desejadas]

In [ ]:
# Salvar o resultado em um arquivo CSV
result.to_csv('/content/drive/MyDrive/Pedro Marketing/2023 12 99Hunters Wrapped/Análise 13.csv', index=False)